In [ ]:
openai_api_key = ""
gigachat_api_key = ""
data_path = 'data/langchain_dataset.csv'

# Устанавливаем необходимые зависимости

In [ ]:
! pip install langchain
! pip install datasets
! pip install faiss-cpu
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 722.6 kB/s eta 0:00:001m938.3 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.8 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.5 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.4 MB/s eta 0:00:0011.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 8.6 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.2 MB/s eta 0:00:0031m11.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.2/613.2 kB 4.8 MB/s eta 0:00:000:00:

In [ ]:
! pip install openai

  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01m1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.4 MB/s eta 0:00:00
Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.12.0
    Uninstalling h11-0.12.0:
      Successfully uninstalled h11-0.12.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.13.7
    Uninstalling httpcore-0.13.7:
      Successfully uninstalled httpcore-0.13.7
  Attempting uninstall: httpx
    Found existing installation: httpx 0.19.0
    Uninstalling httpx-0.19.0:
      Successfully uninstalled httpx-0.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fo

In [ ]:
! pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.5 MB/s eta 0:00:004.4 MB/s eta 0:00:01


# Импортируем библиотеки

In [ ]:
import time
import uuid
import requests
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
import pandas as pd
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts.example_selector import LengthBasedExampleSelector

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Пишем кастомный класс получения токена авторизации для GigaChat

In [ ]:
class GigaChatSecureToken:
    access_token: str
    expires_at: int
    _offset: int = 60  # If token will be expired in {offset} seconds, is_expired() return true

    def __init__(self, access_token: str, expires_at: int):
        self.access_token = access_token
        self.expires_at = expires_at

    def is_expired(self):
        return round(time.time() * 1000) > self.expires_at + self._offset * 1000

# Определяем кастомный класс для работы с API GigaChat

In [ ]:
class GigaChatLLM(LLM):
    api_key: str = None
    temperature: float = 0.7
    secure_token: GigaChatSecureToken = None

    @property
    def _llm_type(self) -> str:
        return "gigachat"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if not self.secure_token or self.secure_token.is_expired():
            print("Obtaining new secure token")
            self._auth()
            if not self.secure_token or self.secure_token.is_expired():
                # New token was not obtained
                print("ERROR: new token was not updated, cannot call LLM")
                return ""
        headers = {
            "Authorization": f"Bearer {self.secure_token.access_token}",
            "Content-Type": "application/json"
        }
        req = {
            "model": "GigaChat:latest",
            "messages": [{
                "role": "user",
                "content": prompt
            }],
            "temperature": self.temperature
        }
        response = requests.post("https://gigachat.devices.sberbank.ru/api/v1/chat/completions", headers=headers, json=req, verify=False)
        if response.status_code != 200:
            print(f"ERROR: LLM call failed, status code: {response.status_code}")
            return ""
        return response.json()["choices"][0]["message"]["content"]

    def _auth(self):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "RqUID": str(uuid.uuid4()),
            "Content-Type": "application/x-www-form-urlencoded"
        }

        scope_info = {"scope": "GIGACHAT_API_PERS"}
        response = requests.post("https://ngw.devices.sberbank.ru:9443/api/v2/oauth", data=scope_info, headers=headers, verify=False)
        if response.status_code != 200:
            print(f"ERROR: Something went wrong while obtaining secure token, status code: {response.status_code}")
            return
        content = response.json()

        expires_at = content["expires_at"]
        token = content["access_token"]
        if not (expires_at and token):
            print("ERROR: server returns empty values for fields 'expires_at' or 'access_token'")
            return
        self.secure_token = GigaChatSecureToken(token, expires_at)


llm = GigaChatLLM(api_key=gigachat_api_key)

In [ ]:
llm(prompt="Пусть a = 5.  Сколько будет a в квадрате")

Obtaining new secure token


'Если мы возьмём число 5 и возведём его в квадрат, то получим 25.'

# Загружаем датасет

In [ ]:
dataframe = pd.read_csv(data_path)

# Для того чтобы эмбеддинг соджержал наибольшее количество полезной информации создадим дополнительный столбец - конкатенацию вопроса и ответа.

# Также для тестирования качества поиска ближайших соседей добавим каждой строке idшник

In [ ]:
dataframe['concatenated'] = dataframe['question'] + ' ' + dataframe['answer']
dataframe = dataframe.assign(id=range(len(dataframe)))

In [ ]:
dataframe

,question,answer,concatenated,id
0,Что такое LangChain и для чего он используется?,LangChain - это фреймворк для разработки прило...,Что такое LangChain и для чего он используется...,0
1,Какие основные части включает в себя фреймворк...,Фреймворк LangChain включает в себя библиотеки...,Какие основные части включает в себя фреймворк...,1
2,Какие ключевые преимущества предлагают пакеты ...,Основные преимущества пакетов LangChain включа...,Какие ключевые преимущества предлагают пакеты ...,2
3,Что такое LCEL в контексте LangChain?,LCEL (LangChain Expression Language) - это дек...,Что такое LCEL в контексте LangChain? LCEL (La...,3
4,Какие модули предоставляет LangChain?,"LangChain предоставляет стандартные, расширяем...",Какие модули предоставляет LangChain? LangChai...,4
...,...,...,...,...
377,Как использовать инструменты с форматом строки...,"from langchain.agents import AgentType, Tool, ...",Как использовать инструменты с форматом строки...,377
378,Как инструменты LangChain определяют схему арг...,"from typing import Any, Dict \n\n from la...",Как инструменты LangChain определяют схему арг...,378
379,Как использовать инструменты LangChain в качес...,from langchain.chat_models import ChatOpenAI ...,Как использовать инструменты LangChain в качес...,379
380,Как LangChain обеспечивает асинхронную поддерж...,import asyncio \n import time \n\n f...,Как LangChain обеспечивает асинхронную поддерж...,380


# Трансформируем датафрейм в датафреймлоадер langchain. page_content_column в нашем случае поле concatenated.

In [ ]:
loader = DataFrameLoader(dataframe, page_content_column = 'concatenated')
data = loader.load()

In [ ]:
data[:2]

[Document(page_content='Что такое LangChain и для чего он используется? LangChain - это фреймворк для разработки приложений, работающих на основе языковых моделей. Он позволяет создавать приложения, которые осведомлены о контексте и могут осуществлять рассуждения на основе предоставленного контекста.', metadata={'question': 'Что такое LangChain и для чего он используется?', 'answer': 'LangChain - это фреймворк для разработки приложений, работающих на основе языковых моделей. Он позволяет создавать приложения, которые осведомлены о контексте и могут осуществлять рассуждения на основе предоставленного контекста.', 'id': 0}),
 Document(page_content='Какие основные части включает в себя фреймворк LangChain? Фреймворк LangChain включает в себя библиотеки LangChain для Python и JavaScript, коллекцию LangChain Templates для развертывания архитектур, LangServe для развертывания цепочек LangChain в виде REST API, и LangSmith - платформу для отладки и тестирования.', metadata={'question': 'Какие

In [ ]:
token_counts_max = dataframe['concatenated'].str.split().str.len().max()
token_counts_max

360

# Определяем модель для эмбеддингов.

In [ ]:
embedding_type = 'openai'
# embedding_type = 'cointegrated/LaBSE-en-ru'

if embedding_type == 'openai':
    openai_batch_size = int(150000 / (3 * token_counts_max))
    embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key, chunk_size = openai_batch_size)
else:
    embeddings = HuggingFaceEmbeddings(model_name=embedding_type)

# Загружаем эмбеддинги в векторную бд

In [ ]:
from langchain import FAISS
from faiss import IndexHNSWFlat
from time import sleep
from tqdm import tqdm

index_name = f'{data_path}_{embedding_type}'

index = IndexHNSWFlat()
faiss = FAISS(embeddings, index, None, None)

try:
    db = FAISS.load_local(index_name, embeddings)
except:
    db = None

if db is None:
    if embedding_type == 'openai':
        for batch_number in tqdm(range(0, len(data), openai_batch_size)):
            batch = data[batch_number: batch_number + openai_batch_size]
            batch_db = faiss.from_documents(batch, embeddings)
            if db is None:
                db = batch_db
            else:
                db.merge_from(batch_db)

            sleep(20)
    else:
        db = faiss.from_documents(data, embeddings)

    db.save_local(index_name)

In [ ]:
query = "Что такое Langchain?"
db.similarity_search(query, k=5)

[Document(page_content='Что такое LangChain и для чего он используется? LangChain - это фреймворк для разработки приложений, работающих на основе языковых моделей. Он позволяет создавать приложения, которые осведомлены о контексте и могут осуществлять рассуждения на основе предоставленного контекста.', metadata={'question': 'Что такое LangChain и для чего он используется?', 'answer': 'LangChain - это фреймворк для разработки приложений, работающих на основе языковых моделей. Он позволяет создавать приложения, которые осведомлены о контексте и могут осуществлять рассуждения на основе предоставленного контекста.', 'id': 0}),
 Document(page_content='Какие ключевые преимущества предлагают пакеты LangChain? Основные преимущества пакетов LangChain включают в себя компоненты для работы с языковыми моделями и готовые цепочки, которые облегчают начало работы и позволяют настраивать существующие цепочки или создавать новые.', metadata={'question': 'Какие ключевые преимущества предлагают пакеты L

# Определяем ретривер - верхнеуровневую обертку над similarity_search, в которую захардкожены какие-то параметры (в нашем случае 5 ближайших соседей)

In [ ]:
retriever = db.as_retriever(search_kwargs = {"k": 5, "score_threshold": 0.3})
retriever.get_relevant_documents(query)

[Document(page_content='Что такое LangChain и для чего он используется? LangChain - это фреймворк для разработки приложений, работающих на основе языковых моделей. Он позволяет создавать приложения, которые осведомлены о контексте и могут осуществлять рассуждения на основе предоставленного контекста.', metadata={'question': 'Что такое LangChain и для чего он используется?', 'answer': 'LangChain - это фреймворк для разработки приложений, работающих на основе языковых моделей. Он позволяет создавать приложения, которые осведомлены о контексте и могут осуществлять рассуждения на основе предоставленного контекста.', 'id': 0}),
 Document(page_content='Какие ключевые преимущества предлагают пакеты LangChain? Основные преимущества пакетов LangChain включают в себя компоненты для работы с языковыми моделями и готовые цепочки, которые облегчают начало работы и позволяют настраивать существующие цепочки или создавать новые.', metadata={'question': 'Какие ключевые преимущества предлагают пакеты L

# Определяем шаблонизатор промпта

In [ ]:
prefix = """
Ответь на вопрос: {query}

При ответе учитывай следующие имеющиеся данные:
"""

context_example_template = """
{context}
"""

suffix = "Если вопрос тесно связан с предоставленными данными, используй их при формировании своего ответа."

# Определяем пайплайн промпт template -> языковая модель

# Объединяем все в одну функцию

In [ ]:
from typing import Tuple

def escape_f_string(text):
  return text.replace('{', '{{').replace('}', '}}')

def escape_examples(examples):
    return [{k: escape_f_string(v) for k, v in example.items()} for example in examples]


def get_answer(question: str) -> Tuple[str, str]:
  res = retriever.get_relevant_documents(question)
  examples = [{"context": doc.page_content } for doc in res ]

  context_example_template_prompt = PromptTemplate(
      input_variables=["context"],
      template=context_example_template
  )

  example_selector = LengthBasedExampleSelector(
    examples=escape_examples(examples),
    example_prompt=context_example_template_prompt,
    max_length=900
  )

  few_shot_prompt_template_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt=context_example_template_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
  )

  chain = few_shot_prompt_template_prompt | llm

  answer = chain.invoke({ "query": question })


  prompt = few_shot_prompt_template_prompt.format(query = question)
  return answer, prompt

In [ ]:
answer, prompt = get_answer(query)
print(prompt)
print(answer)


Ответь на вопрос: Что такое Langchain?

При ответе учитывай следующие имеющиеся данные:


Что такое LangChain и для чего он используется? LangChain - это фреймворк для разработки приложений, работающих на основе языковых моделей. Он позволяет создавать приложения, которые осведомлены о контексте и могут осуществлять рассуждения на основе предоставленного контекста.


Какие ключевые преимущества предлагают пакеты LangChain? Основные преимущества пакетов LangChain включают в себя компоненты для работы с языковыми моделями и готовые цепочки, которые облегчают начало работы и позволяют настраивать существующие цепочки или создавать новые.


Какова роль LangChain в экосистеме инструментов? LangChain является частью богатой экосистемы инструментов, которые интегрируются с этим фреймворком и строятся на его основе, включая различные интеграции и руководства по лучшим практикам разработки.


Какие интеграции предлагает LangChain и как они способствуют созданию приложений? LangChain предлагает

# Сравним аугментированную модель с неаугментированной

In [ ]:
questions = [
    "Как подключить векторную бд в Langchain?",
    "Что такое ретривер в контексте Langchain?",
    "Как объединить несколько шагов в цепочку в LangChain?",
    "Как в Langchain работать с OpenAiEmbeddings?",
    "Как использовать CSVLoader в Langchain?",
    "Как мне в CSVLoader в Langchain указать свой путь до файла если мой файл лежит в /home/user/data.csv?",
    "Почему трава зеленая?"
]

In [ ]:
compare_df = pd.DataFrame({"gigachat": [], "gigachat + augmentation": [], "prompt": []})
i = 0
for question in tqdm(questions):
    llm_answer = llm(question)
    llm_augmented_answer, llm_augmented_prompt = get_answer(question)

    concat_df = pd.DataFrame.from_dict({"gigachat": [llm_answer], "gigachat + augmentation": [llm_augmented_answer], "prompt": [llm_augmented_prompt]})

    compare_df = pd.concat([compare_df, concat_df], ignore_index = True)
    if embedding_type == 'openai' and i < len(questions) - 1:
        sleep(20)

    i += 1

 14%|████████████████████████████████████████▏                                                                                                                                                                                                                                                | 1/7 [00:35<03:30, 35.10s/it]

Obtaining new secure token


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [03:28<00:00, 29.73s/it]


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
compare_df

,gigachat,gigachat + augmentation,prompt
0,"Для подключения векторной базы данных (VDB) в Langchain необходимо выполнить следующие шаги:\n\n1. Установить и настроить VDB на сервере базы данных.\n2. Создать таблицу в VDB, содержащую необходимые данные для работы приложения.\n3. Настроить соединение между приложением и VDB. Для этого необходимо указать параметры соединения, такие как хост, порт, имя пользователя и пароль.\n4. Настроить параметры доступа к данным в VDB. Это может включать в себя установку прав доступа к таблицам и полям, а также настройку шифрования данных.\n5. Настроить параметры безопасности соединения. Это может включать в себя установку SSL-сертификата и настройку параметров авторизации и аутентификации.\n6. Проверить работоспособность соединения и убедиться, что приложение успешно работает с данными из VDB.\n\nДля более подробной информации о настройке соединения с VDB рекомендуется обратиться к документации конкретной системы управления базами данных, используемой в VDB.","Для подключения векторной базы данных в LangChain необходимо использовать модуль 'Data Connection'. Этот модуль предоставляет функциональность для загрузки документов, их преобразования, создания векторных представлений и организации эффективного поиска и извлечения информации.\n\nДля интеграции с 'Data Connection' в LangChain необходимо выполнить следующие шаги:\n\n1. Установить модуль 'Data Connection' в LangChain. Для этого можно воспользоваться командой 'pip install -e .' из исходного кода.\n\n2. Создать объект 'DataConnection' и настроить его параметры, такие как источник данных, формат файла и т.д.\n\n3. Подключить векторную базу данных к объекту 'DataConnection'. Для этого необходимо указать путь к базе данных и настроить параметры соединения, такие как имя пользователя, пароль и т.д.\n\n4. Использовать полученный объект 'DataConnection' для загрузки данных, создания векторных представлений и организации поиска и извлечения информации.\n\nИнтеграция с 'Data Connection' в LangChain позволяет расширить возможности LLM, интегрируя пользовательские данные и контекст в процесс генерации ответов. Это улучшает релевантность и точность ответов, позволяя моделям языка адаптироваться к конкретным потребностям пользователя и обеспечивая более глубокое понимание контекста запросов.","\nОтветь на вопрос: Как подключить векторную бд в Langchain?\n\nПри ответе учитывай следующие имеющиеся данные:\n\n\nЧто такое модуль 'Data Connection' в LangChain? Модуль 'Data Connection' в LangChain предоставляет функциональность для подключения к различным источникам данных и их использования в контексте LLM (Large Language Models). Это включает в себя инструменты для загрузки документов, их преобразования, создания векторных представлений и организации эффективного поиска и извлечения информации.\n\n\nКакие преимущества предоставляет интеграция с 'Data Connection' в LangChain? Интеграция с 'Data Connection' в LangChain позволяет расширить возможности LLM, интегрируя пользовательские данные и контекст в процесс генерации ответов. Это улучшает релевантность и точность ответов, позволяя моделям языка адаптироваться к конкретным потребностям пользователя и обеспечивая более глубокое понимание контекста запросов.\n\n\nКакие интеграции предлагает LangChain и как они способствуют созданию приложений? LangChain предлагает обширную экосистему интеграций с различными внешними ресурсами, такими как локальные и удаленные файловые системы, API и базы данных. Эти интеграции позволяют разработчикам создавать гибкие приложения, сочетающие возможности языковых моделей (LLM) с доступом, взаимодействием и манипулированием внешними ресурсами.\n\n\nКакова цель объединения агентов и векторных хранилищ в LangChain? Цель объединения агентов и векторных хранилищ в LangChain - создать агента, который может взаимодействовать с данными, индексированными в векторном хранилище, и обеспечивать агентное взаимодействие для поиска и извлечения данных&#8203;``【oaicite:2】``&#8203;.\n\n\nКак уст

In [ ]:
compare_df.to_html('final_work_result.html')

# Посмотрим качество наших эмбеддингов - насколько они правильно находят ответ по запросу

In [ ]:
top_n = 5
retriever = db.as_retriever(search_kwargs = {"k": top_n, "score_threshold": 0.3})

test_subset_len = 20
doc_length = min(len(data), test_subset_len)

# Посмотрим accuracy когда мы задаем точно такие же вопросы, котоые есть в датасете

In [ ]:
relevant_in_top_n = 0

for doc in tqdm(data[:test_subset_len]):
    question = doc.metadata['question']
    question_id = doc.metadata['id']

    relevant_documents = retriever.get_relevant_documents(question)
    relevant_document_ids = map(lambda x: x.metadata['id'], relevant_documents)
    if question_id in relevant_document_ids:
        relevant_in_top_n += 1

    sleep(20)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [06:53<00:00, 20.67s/it]


In [ ]:
print(f'Точность на тех же вопросах {relevant_in_top_n / doc_length}')

Точность на тех же вопросах 1.0


# Попроисим gigachat переформулировать наши вопросы и проверим точность на них.

In [ ]:
relevant_in_top_n = 0
question_dict = {
    'questions': [],
    'rephrased_questions': []
}


for doc in tqdm(data[:test_subset_len]):
    question = doc.metadata['question']
    question_id = doc.metadata['id']

    rephrased_question = llm(f'Переформулируй этот вопрос: {question}')

    question_dict['questions'].append(question)
    question_dict['rephrased_questions'].append(rephrased_question)

    relevant_documents = retriever.get_relevant_documents(rephrased_question)
    relevant_document_ids = map(lambda x: x.metadata['id'], relevant_documents)
    if question_id in relevant_document_ids:
        relevant_in_top_n += 1

    sleep(20)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [07:22<00:00, 22.11s/it]


In [ ]:
pd.DataFrame.from_dict(question_dict)

,questions,rephrased_questions
0,Что такое LangChain и для чего он используется?,Для чего предназначен LangChain и что это за технология?
1,Какие основные части включает в себя фреймворк LangChain?,Какие компоненты входят в состав фреймворка LangChain?
2,Какие ключевые преимущества предлагают пакеты LangChain?,Какие основные преимущества предоставляют пакеты LangChain?
3,Что такое LCEL в контексте LangChain?,"В контексте LangChain, что означает LCEL?"
4,Какие модули предоставляет LangChain?,Какие функциональные возможности предлагает LangChain?
5,Какие примеры использования LangChain приведены в документации?,В документации приведены какие примеры использования LangChain?
6,Какова роль LangChain в экосистеме инструментов?,Какую роль играет LangChain в экосистеме инструментов?
7,Как установить LangChain с использованием Pip и Conda?,Как можно установить пакет LangChain с помощью инструментов Pip и Conda?
8,Как установить LangChain из исходного кода?,Как можно установить LangChain из исходного кода?
9,Что представляет собой пакет 'langchain-experimental' и как его установить?,Какой пакет называется 'langchain-experimental' и как его можно установить?


In [ ]:
print(f'Точность на переформулированных вопросах {relevant_in_top_n / doc_length}')

Точность на переформулированных вопросах 1.0


# Как видим, на сабсемпле данных эмбеддинг одинакого хорошо работает как на точных вопросах, так и на перефразированных.